In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense,LeakyReLU,BatchNormalization,Dropout,Concatenate,Input
from tensorflow.keras.optimizers import SGD, Adam
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("./drive/MyDrive/SpikeProteinFragmented.csv")

In [ ]:
df

,Unnamed: 0,Seq,Var
0,0,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,alpha
1,1,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,alpha
2,2,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,alpha
3,3,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,alpha
4,4,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,alpha
...,...,...,...
2232,2232,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,omicron
2233,2233,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,omicron
2234,2234,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,omicron
2235,2235,MFVFL FVFLV VFLVL FLVLL LVLLP VLLPL LLPLV LPLV...,omicron


In [ ]:
x=df["Seq"]
x_encoded = [one_hot(d,1276) for d in x]

In [ ]:
max_length = len(max(x_encoded,key=len))+1
max_length 

1276

In [ ]:
x_padded = pad_sequences(x_encoded, maxlen=max_length, padding='post')
print(x_padded)

[[1052  972  348 ...    0    0    0]
 [1052  972  348 ...    0    0    0]
 [1052  972  348 ...    0    0    0]
 ...
 [1052  972  348 ...    0    0    0]
 [1052  972  348 ...    0    0    0]
 [1052  972  348 ...    0    0    0]]


In [ ]:
le = preprocessing.LabelEncoder()
y=df["Var"].values
y_labeled=le.fit_transform(y)
y_labeled

array([ 0,  0,  0, ..., 10, 10, 10])

In [ ]:
from sklearn.preprocessing import StandardScaler
# load data 
# create scaler
scaler = StandardScaler()
# fit and transform in one step
x_scaled = scaler.fit_transform(x_padded)
x_scaled

array([[ 0.13665525,  0.36077412, -0.38465538, ..., -0.02991412,
        -0.02991412,  0.        ],
       [ 0.13665525,  0.36077412, -0.38465538, ..., -0.02991412,
        -0.02991412,  0.        ],
       [ 0.13665525,  0.36077412, -0.38465538, ..., -0.02991412,
        -0.02991412,  0.        ],
       ...,
       [ 0.13665525,  0.36077412, -0.38465538, ..., -0.02991412,
        -0.02991412,  0.        ],
       [ 0.13665525,  0.36077412, -0.38465538, ..., -0.02991412,
        -0.02991412,  0.        ],
       [ 0.13665525,  0.36077412, -0.38465538, ..., -0.02991412,
        -0.02991412,  0.        ]])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test  =  train_test_split(x_scaled, y_labeled,test_size=0.2, random_state=4)
len(x_train)

1789

In [ ]:
x_train.shape

(1789, 1276)

In [ ]:
lat=1000

In [ ]:
#Parent Sequence Encoder
par_inputs = Input(shape=(None,))
enc_embedding = Embedding(lat,500)
par_inputs_emb=enc_embedding(par_inputs)

enc_BiLSTM = Bidirectional(LSTM(500,return_state=True))
enc_outputs, fwd_hPar, fwd_cPar, bwd_hPar, bwd_cPar = enc_BiLSTM(par_inputs_emb)
state_hPar = Concatenate()([fwd_hPar, bwd_hPar])
state_cPar= Concatenate()([fwd_cPar, bwd_cPar])
encoder_statePar = [state_hPar, state_cPar]
ParentEncoder_model = Model([par_inputs], encoder_statePar) 

In [ ]:
#Generated Sequence Encoder
gen_inputs = Input(shape = (None,lat))
enc_inputsGen = Dense(500, activation = 'linear', use_bias = False)(gen_inputs)
enc_outputsGen, fwd_hGen, fwd_cGen, bwd_hGen, bwd_cGen = enc_BiLSTM(enc_inputsGen)
state_hGen = Concatenate()([fwd_hGen, bwd_hGen])
state_cGen = Concatenate()([fwd_cGen, bwd_cGen])
encoder_stateGen = [state_hGen, state_cGen]
GenEncoder_model = Model([gen_inputs], encoder_stateGen) 
GenEncoder_model

In [ ]:
z = Input(shape = (None,lat))
z

<KerasTensor: shape=(None, None, 1000) dtype=float32 (created by layer 'input_3')>

In [ ]:
xPar = ParentEncoder_model([par_inputs])
xGen = GenEncoder_model ([gen_inputs])
xConcat = Concatenate()(xPar+xGen)
x = Dropout(0.2)(xConcat)
x = BatchNormalization()(x)
x = Dense(128)(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(64)(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
output_class = Dense(1, activation = 'tanh')(x)
Discriminator = Model([par_inputs, gen_inputs], outputs = [output_class]) 

In [ ]:
Discriminator.compile( loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

In [ ]:
Discriminator.trainable = False

In [ ]:
fake_pred = Discriminator([x_train,z])

In [ ]:
combined_model_gen = Model(z, fake_pred)
# Compile the combined model
combined_model_gen.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5))

In [ ]:
batch_size = x_train.shape[0]
epochs = 30000
sample_period = 200 # every `sample_period` steps generate and save some data"
# Create batch labels to use when calling train_on_batch

# Store the losses
d_losses = []
g_losses = []
ones= np.ones(batch_size)
zeros= np.zeros(batch_size)


In [ ]:
  # Generate fake images
noise = np.random.randn(batch_size,x_train.shape[1], lat)
fake_seq = GenEncoder_model.predict(noise)

In [ ]:
d_loss_real, d_acc_real = Discriminator.fit([x_train,noise], ones)


In [ ]:
d_loss_fake, d_acc_fake = Discriminator.fit([fake_seq,noise], zeros)

In [ ]:

def sample_seq(epoch):
  rows, cols = 1,1276
  noise = np.random.randn(rows * cols)
  seq = GenEncoder_model.predict(noise)

  # Rescale images 0 - 1
  newseq = scaler.fit_transform(seq)
  print(seq)
### Train the GAN
# Main training loop
for epoch in range(epochs):
  ###########################
  ### Train discriminator ###
  ###########################
  
  # Select a random batch of images
  idx = np.random.randint(0, x_train.shape[0], batch_size)
  real_seq = x_train[idx]
  
  # Generate fake images
  noise = np.random.randn(batch_size,x_train.shape[1], lat)
  fake_seq = GenEncoder_model.predict(noise)
  
  # Train the discriminator
  # both loss and accuracy are returned
  d_loss_real, d_acc_real = Discriminator.train_on_batch(real_seq, ones)
  d_loss_fake, d_acc_fake = Discriminator.train_on_batch([fake_seq,z], zeros)
  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc  = 0.5 * (d_acc_real + d_acc_fake)
  
  
  #######################
  ### Train generator ###
  #######################
  
  noise = np.random.randn(batch_size, lat)
  g_loss = combined_model_gen.train_on_batch(noise, ones)
  
  # do it again!
  noise = np.random.randn(batch_size, lat)
  g_loss = combined_model_gen.train_on_batch(noise, ones)
  
  # Save the losses
  d_losses.append(d_loss)
  g_losses.append(g_loss)
  
  if epoch % 100 == 0:
    print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, \
      d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
  
  if epoch % sample_period == 0:
    sample_seq(epoch)